# OnePanel AutoML 0.1a

AutoML is a framework that allows building automated machine learning pipelines easily and declaratively, running them locally (current implementation) or on the cluster (TBD).

The framework can be easily extened with new features.
Currently AutoML is integrated with popular opensource machine learning libraries like Scikit-learn and Hyperopt

In [1]:
import sys
# AutoML uses Python's logging module
import logging

# Various sklearn models and metrics
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from xgboost.sklearn import XGBClassifier

# AutoML Clasees
from automl.pipeline import LocalExecutor, Pipeline, PipelineStep, PipelineData
from automl.data.dataset import Dataset
from automl.model import ModelSpace, CV, Validate, ChooseBest
from automl.hyperparam.templates import (random_forest_hp_space, 
                                         knn_hp_space, svc_kernel_hp_space, 
                                         grad_boosting_hp_space, 
                                         xgboost_hp_space)
from automl.feature.generators import FormulaFeatureGenerator
from automl.feature.selector import FeatureSelector
from automl.hyperparam.optimization import Hyperopt
from automl.combinators import RandomChoice

logging.basicConfig(level=logging.INFO)
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

# Core concepts
AutoML follows code-is-data and data-is-code philosophy. You defind automated machine learning pipelines as data structures that can be executed later.
Key concepts in AutoML are 
* `Pipeline` - a machine learning pipeline. It executes various steps inside the pipeline passing each step output as an input to the next step
* `PipelineStep` - all `Pipeline`s consist of steps. AutoML provide lots of different steps out of the box
* `Executor` - executes a pipeline. Currently AutoML provides `LocalExecutor` which runs pipeline locally. Future versions will have `DistributedExecutor` built-in 

AutoML can easily be extended by implementing own `PipelineStep`s. Next, we will use various built-in `PipelineStep`s to create an automated classification pipeline.

In [2]:
# Let's create a dataset first
x, y = make_classification(
      n_samples=1000,
      n_features=40,
      n_informative=2,
      n_redundant=10,
      flip_y=0.05)

# We will use AutoML Dataset class to wrap our data 
# into structure that can be understanded by AutoML
data = Dataset(x, y)

# Next, we define our ModelSpace. ModelSpace is initialized by a list of tuples.
# First element of each tuple should be an sklearn-like estimator with fit method
# The second one is model parameter dictionary. Here we do not define parameters 
# explicitly, but use hyperparameter templates from AutoML. Those templates can be
# used later by Hyperopt step to find best model parameters automatically
model_list = [
      (RandomForestClassifier, random_forest_hp_space()),
      (KNeighborsClassifier, knn_hp_space(lambda key: key)),
      (XGBClassifier, xgboost_hp_space())
  ]


# Create executor, initialize it with our classification dataset 
# and set total number of epochs to 2 (the pipeline will be run two times in a row).
# We can load any pipeline into executor using << operator like below:
context, pipeline_data = LocalExecutor(data, epochs=2) << \
    (Pipeline() # Here we define the pipeline. Steps can be added to pipeline using >> operator
     # First we define our ModelSpace. We wrap it with PipelineStep class 
     # and set initializer=True so that ModelSpace step will be run only at the first epoch
     >> PipelineStep('model space', ModelSpace(model_list), initializer=True)
     # But we are not obliged to wrap all steps with PipelineStep.
     # This will be done automatically if we do not need to set any special parameters 
     # We use FormulaFeatureGenerator to create arithmetic combinations of features from the dataset
     >> FormulaFeatureGenerator(['+', '-', '*']) 
     # Next we use Hyperopt to find the best combination of hyperparameters for each model
     # We use test set validation with ROC AUC metric as a score function.
     # CV could be used instead of Validate to perform cross-validation
     >> Hyperopt(Validate(test_size=0.1, metrics=roc_auc_score), max_evals=5) 
     # Then we choose the best performing model we found
     >> ChooseBest(1)
     # And select 10 best features
     >> FeatureSelector(10))

for result in pipeline_data.return_val:
    print(result.model, result.score)
print(pipeline_data.dataset.data.shape)

LocalExecutor - INFO - Framework version: v0.1.4
LocalExecutor - INFO - Starting AutoML Epoch #1
LocalExecutor - INFO - Dataset columns: ['base_feature_0', 'base_feature_1', 'base_feature_2', 'base_feature_3', 'base_feature_4', 'base_feature_5', 'base_feature_6', 'base_feature_7', 'base_feature_8', 'base_feature_9', 'base_feature_10', 'base_feature_11', 'base_feature_12', 'base_feature_13', 'base_feature_14', 'base_feature_15', 'base_feature_16', 'base_feature_17', 'base_feature_18', 'base_feature_19', 'base_feature_20', 'base_feature_21', 'base_feature_22', 'base_feature_23', 'base_feature_24', 'base_feature_25', 'base_feature_26', 'base_feature_27', 'base_feature_28', 'base_feature_29', 'base_feature_30', 'base_feature_31', 'base_feature_32', 'base_feature_33', 'base_feature_34', 'base_feature_35', 'base_feature_36', 'base_feature_37', 'base_feature_38', 'base_feature_39']
  0%|          | 0/5 [00:00<?, ?it/s]LocalExecutor - INFO - Running step 'model space'
LocalExecutor - INFO - Ru


  █████╗ ██╗   ██╗████████╗ ██████╗ ███╗   ███╗██╗     
 ██╔══██╗██║   ██║╚══██╔══╝██╔═══██╗████╗ ████║██║     
 ███████║██║   ██║   ██║   ██║   ██║██╔████╔██║██║     
 ██╔══██║██║   ██║   ██║   ██║   ██║██║╚██╔╝██║██║     
 ██║  ██║╚██████╔╝   ██║   ╚██████╔╝██║ ╚═╝ ██║███████╗
 ╚═╝  ╚═╝ ╚═════╝    ╚═╝    ╚═════╝ ╚═╝     ╚═╝╚══════╝



/Users/rush/Library/Python/3.6/lib/python/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
hyperopt.tpe - INFO - tpe_transform took 0.003790 seconds
hyperopt.tpe - INFO - TPE using 2/2 trials with best loss 0.049495
hyperopt.tpe - INFO - tpe_transform took 0.003041 seconds
hyperopt.tpe - INFO - TPE using 3/3 trials with best loss 0.038384
/Users/rush/Library/Python/3.6/lib/python/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
hyperopt.tpe - INFO - tpe_transform took 0.003705 seconds
hyperopt.tpe - INFO - TPE using 4/4 trials with best loss 0.038384
/Users/rush/Library/Python/3.6/lib/python/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSync

Hyperopt - INFO - Running hyperparameter optimization for <class 'sklearn.ensemble.forest.RandomForestClassifier'>
hyperopt.tpe - INFO - tpe_transform took 0.003914 seconds
hyperopt.tpe - INFO - TPE using 0 trials
hyperopt.tpe - INFO - tpe_transform took 0.003552 seconds
hyperopt.tpe - INFO - TPE using 1/1 trials with best loss 0.058586
hyperopt.tpe - INFO - tpe_transform took 0.003915 seconds
hyperopt.tpe - INFO - TPE using 2/2 trials with best loss 0.038384
hyperopt.tpe - INFO - tpe_transform took 0.003564 seconds
hyperopt.tpe - INFO - TPE using 3/3 trials with best loss 0.038384
hyperopt.tpe - INFO - tpe_transform took 0.004145 seconds
hyperopt.tpe - INFO - TPE using 4/4 trials with best loss 0.038384
Hyperopt - INFO - Reversing best score bask to original form as reverse_score=True
Hyperopt - INFO - {'n_neighbors': <hyperopt.pyll.base.Apply object at 0x10c8e0198>, 'weights': <hyperopt.pyll.base.Apply object at 0x10c8e02b0>, 'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean

XGBClassifier(base_score=0.5, booster='gbtree',
       colsample_bylevel=0.7787641157267267,
       colsample_bytree=0.5478702825933608, gamma=0.020806587332124784,
       learning_rate=0.054724673451827605, max_delta_step=0, max_depth=2,
       min_child_weight=78, missing=None, n_estimators=5600, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.22576345473949086, reg_lambda=1.1010506375839129,
       scale_pos_weight=1, seed=3, silent=True,
       subsample=0.7180595467378208) 0.9797979797979798
(1000, 11)


# Extending AutoML

First, let's look at how `PipelineStep`s can be created by creating a simple hello world pipeline.

In [3]:
# Let's create a simple pipeline
pipeline = Pipeline() >> PipelineStep('hello_step', lambda inp, context: print("Hello!"))

# And execute it locally
LocalExecutor() << pipeline

LocalExecutor - INFO - Framework version: v0.1.4
LocalExecutor - INFO - Starting AutoML Epoch #1
100%|██████████| 1/1 [00:00<00:00, 496.43it/s]


  █████╗ ██╗   ██╗████████╗ ██████╗ ███╗   ███╗██╗     
 ██╔══██╗██║   ██║╚══██╔══╝██╔═══██╗████╗ ████║██║     
 ███████║██║   ██║   ██║   ██║   ██║██╔████╔██║██║     
 ██╔══██║██║   ██║   ██║   ██║   ██║██║╚██╔╝██║██║     
 ██║  ██║╚██████╔╝   ██║   ╚██████╔╝██║ ╚═╝ ██║███████╗
 ╚═╝  ╚═╝ ╚═════╝    ╚═╝    ╚═════╝ ╚═╝     ╚═╝╚══════╝

Hello!


(<automl.pipeline.PipelineContext at 0x10756a828>, None)

As you can see steps can be added to a pipeline using `>>` operator. A pipeline may contain any number of steps. Any `PipelineStep` is constructed by passing a step name and a `callable` which will be executed when `Pipeline` is run by an `Executor`. It's important to mention that all `Pipeline`s are lazy and all steps inside will be executed only when `Pipeline` is loaded into `Executor.`

`PipelineStep` syntax is pretty verbose, but it can be simplified. You can pass any `callable` to a pipeline and it will be wrapped into `PipelineStep` automatically. Step function should have two arguments: `input` and `context`. `input` must be loaded through executor parameters, `context` contains global variables, available for each step. If `PipelineStep` returns any value, it should wrap it into `PipelineData` class. `input` passed to an `Executor` is wrapped to `PipelineData` automatically

In [4]:
# We create two steps that add 1 and 2 to input data
plus_one = PipelineStep('plus_one', lambda inp, context: inp.dataset + 1)
plus_two = PipelineStep('plus_two', lambda inp, context: inp.dataset + 2)

LocalExecutor(0) << \
    (Pipeline()
     # We use RandomChoice combinator to choose randomly between two steps while executing the pipeline
     >> RandomChoice([plus_one, plus_two]))

LocalExecutor - INFO - Framework version: v0.1.4
LocalExecutor - INFO - Starting AutoML Epoch #1
100%|██████████| 1/1 [00:00<00:00, 1105.22it/s]


  █████╗ ██╗   ██╗████████╗ ██████╗ ███╗   ███╗██╗     
 ██╔══██╗██║   ██║╚══██╔══╝██╔═══██╗████╗ ████║██║     
 ███████║██║   ██║   ██║   ██║   ██║██╔████╔██║██║     
 ██╔══██║██║   ██║   ██║   ██║   ██║██║╚██╔╝██║██║     
 ██║  ██║╚██████╔╝   ██║   ╚██████╔╝██║ ╚═╝ ██║███████╗
 ╚═╝  ╚═╝ ╚═════╝    ╚═╝    ╚═════╝ ╚═╝     ╚═╝╚══════╝



(<automl.pipeline.PipelineContext at 0x10d142780>, 1)

It is recommended to create complex callables for `PipelineStep`s as classes:

In [5]:
class ComplexStep:
    def __init__(self):
        print("Initializing ComplexStep")
        
    def __call__(self, inp, context):
        print(inp)
        return inp
    
LocalExecutor() << (Pipeline() >> ComplexStep())

LocalExecutor - INFO - Framework version: v0.1.4
LocalExecutor - INFO - Starting AutoML Epoch #1
100%|██████████| 1/1 [00:00<00:00, 722.53it/s]

Initializing ComplexStep

  █████╗ ██╗   ██╗████████╗ ██████╗ ███╗   ███╗██╗     
 ██╔══██╗██║   ██║╚══██╔══╝██╔═══██╗████╗ ████║██║     
 ███████║██║   ██║   ██║   ██║   ██║██╔████╔██║██║     
 ██╔══██║██║   ██║   ██║   ██║   ██║██║╚██╔╝██║██║     
 ██║  ██║╚██████╔╝   ██║   ╚██████╔╝██║ ╚═╝ ██║███████╗
 ╚═╝  ╚═╝ ╚═════╝    ╚═╝    ╚═════╝ ╚═╝     ╚═╝╚══════╝



(<automl.pipeline.PipelineContext at 0x10d142f98>,
 <automl.pipeline.PipelineData at 0x10d142358>)